In [40]:
import pandas as pd

In [41]:
# set the directory to where the data is
import os

os.chdir(r"D:\Gene_Project")

In [42]:
unsupervised_data = pd.read_csv(r"all_unsupervised_genes.csv")

In [43]:
unsupervised_data.head()

,sequence
0,"1.A.72,MerR,GH23"
1,"3.A.1,CE4"
2,"3.A.23,9.A.5,MCPsignal,2.A.21,2.A.22,TetR_N,3...."
3,"CE4,8.A.5,3.A.1,3.A.1,3.A.1"
4,"GT51,Peripla_BP_2,3.A.1,3.A.1,9.B.169"


In [44]:
unsupervised_data.drop_duplicates()

,sequence
0,"1.A.72,MerR,GH23"
1,"3.A.1,CE4"
2,"3.A.23,9.A.5,MCPsignal,2.A.21,2.A.22,TetR_N,3...."
3,"CE4,8.A.5,3.A.1,3.A.1,3.A.1"
4,"GT51,Peripla_BP_2,3.A.1,3.A.1,9.B.169"
...,...
240617,"3.A.23,GH23,1.B.1,Sigma70_r2|Sigma70_r4_2,FecR..."
240618,"3.A.1,3.A.1,3.A.1,3.A.1,1.B.48,Pyr_redox_2,1.B..."
240619,"9.B.145,HTH_1|LysR_substrate,2.A.1,3.A.1,3.A.1..."
240620,"1.B.12,Peripla_BP_3|LacI,3.A.1,SBP_bac_1,3.A.1..."


In [45]:
# unsupervised_data[unsupervised_data.duplicated(keep=False)]

In [46]:
import py_stringmatching

In [47]:
from py_stringmatching.similarity_measure.monge_elkan import MongeElkan

In [48]:
# load the data - new data that was provided
supervised_data = pd.read_csv(r"pul_seq_low_high_substr_year_corrected.csv")

In [49]:
supervised_data

,PULid,sig_gene_seq,low_level_substr,high_level_substr,Pub_year
0,PUL0001,"GH1,8.A.49,CE2,GH130,GH130,3.A.1,3.A.1,SBP_bac...",beta-mannan,beta-mannan,2019
1,PUL0002,GH16,lichenan,beta-glucan,1996
2,PUL0003,"GH30_8,GH43_16|CBM6",xylan,xylan,2016
3,PUL0004,"4.A.1,GH1","glucose,cellobiose,maltose",multiple_substrates,2016
4,PUL0005,"GH94,GH3","beta-glucan,sophorose,laminaribiose",multiple_substrates,2016
...,...,...,...,...,...
649,PUL0652,"GH2,2.A.21,3.D.1,GH96,GH96,2.A.1,GH50,GH50,GH5...",agar,algal glycans,2021
650,PUL0653,"GH16_16,HTH_8,PfkB,GH2,GH117|GH117,3.A.1,3.A.1...",agar,algal glycans,2021
651,PUL0654,"GerE,1.B.14,GH144,GH144,CE1,GH3,BLUF,PL6|PL6_1...",alginate,algal glycans,2021
652,PUL0655,"1.B.14,PL7,PL7,GntR",alginate,algal glycans,2021


In [50]:
supervised_data[supervised_data.duplicated(keep=False)]

,PULid,sig_gene_seq,low_level_substr,high_level_substr,Pub_year


In [51]:
# supervised_data[supervised_data["high_level_substr"].isin(["alpha-glucan", "beta-mannan"])]

In [52]:
supervised_data[supervised_data["high_level_substr"] == "multiple_substrates"]["low_level_substr"].value_counts()

O-glycan,N-glycan                                                21
mucin,O-glycan                                                    6
lactose,host glycan                                               6
carboxymethylcellulose,xylan,beta-glucan,lichenan                 6
sucrose,kestose,nystose                                           5
                                                                 ..
beta-glucan,sophorose,laminaribiose                               1
cellulose,xylan,xyloglucan,pectin,mannan                          1
xylan,glucomannan,plant polysaccharide                            1
O-antigen,unknown polysaccharide                                  1
unsaturated hyaluronate disaccharide,chondroitin disaccharide     1
Name: low_level_substr, Length: 73, dtype: int64

In [53]:
supervised_data["low_level_substr"].value_counts()[:40]

capsule polysaccharide                               60
O-antigen                                            37
pectin                                               28
xylan                                                25
alginate                                             21
O-glycan,N-glycan                                    21
cellobiose                                           14
chitin                                               14
beta-glucan                                          12
cellulose                                            12
N-glycan                                             12
sucrose                                              12
lactose                                              11
galactan                                             10
arabinan                                             10
exopolysaccharide                                     9
alpha-mannan                                          8
carrageenan                                     

In [54]:
supervised_strings = supervised_data["sig_gene_seq"]

In [55]:
a, b = supervised_strings[0], supervised_strings[1]

In [56]:
a.split(",")

['GH1',
 '8.A.49',
 'CE2',
 'GH130',
 'GH130',
 '3.A.1',
 '3.A.1',
 'SBP_bac_1',
 'LacI',
 'GH36',
 'GH113']

In [57]:
def pair_sim(a, b): 
    me = MongeElkan()
    split_a = a.split(",")
    split_b = b.split(",")
    score = me.get_raw_score(split_a, split_b)
    return score

In [58]:
pair_sim(a,b)

0.3957575830546292

In [59]:
from joblib import Parallel, delayed

In [60]:
supervised_strings[:5].values

array(['GH1,8.A.49,CE2,GH130,GH130,3.A.1,3.A.1,SBP_bac_1,LacI,GH36,GH113',
       'GH16', 'GH30_8,GH43_16|CBM6', '4.A.1,GH1', 'GH94,GH3'],
      dtype=object)

In [61]:
positions = [[i,j] for i in range(0,len(supervised_strings)) for j in range(0,len(supervised_strings))]

In [62]:
similarities = Parallel(n_jobs=6, verbose = 5)(delayed(pair_sim)(i, j) for i in supervised_strings for j in supervised_strings)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 372 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 27636 tasks      | elapsed:    2.1s
[Parallel(n_jobs=6)]: Done 92148 tasks      | elapsed:    7.3s
[Parallel(n_jobs=6)]: Done 175092 tasks      | elapsed:   13.3s
[Parallel(n_jobs=6)]: Done 276468 tasks      | elapsed:   20.8s
[Parallel(n_jobs=6)]: Done 396276 tasks      | elapsed:   29.7s
[Parallel(n_jobs=6)]: Done 427716 out of 427716 | elapsed:   32.2s finished


In [63]:
import numpy as np

In [64]:
similarities_matrix = np.array(similarities).reshape(654,654)

In [65]:
similarities_matrix

array([[1.        , 0.39575758, 0.5476936 , ..., 0.8629798 , 0.53170033,
        0.61272728],
       [0.94166666, 1.        , 0.82222223, ..., 0.84833336, 0.55000001,
        0.95999998],
       [0.84      , 0.75555557, 1.        , ..., 0.84166667, 0.51249999,
        0.71555558],
       ...,
       [0.67099916, 0.3686633 , 0.43025589, ..., 1.        , 0.63345118,
        0.48720034],
       [0.58263889, 0.25555556, 0.21458333, ..., 0.90303031, 1.        ,
        0.28263889],
       [0.73555557, 0.66907406, 0.61185184, ..., 0.80888889, 0.55518518,
        1.        ]])

In [66]:
from sklearn.manifold import TSNE

In [67]:
X_embedded = TSNE(n_components=3, learning_rate='auto').fit_transform(similarities_matrix)

C:\Users\vedpi\anaconda3\envs\newone\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.



In [68]:
X_embedded.shape

(654, 3)

In [69]:
tsne_data = pd.concat([pd.DataFrame(X_embedded), pd.DataFrame(supervised_data["high_level_substr"])],1)

C:\Users\vedpi\AppData\Local\Temp/ipykernel_18312/1079212839.py:1: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only



In [70]:
tsne_data.to_csv("tsne_3d.csv", index = False)

In [71]:
import plotly.express as px

In [72]:
fig = px.scatter_3d(tsne_data, x=0, y=1, z=2, color = "high_level_substr",
#                     color_discrete_map={'pectin':'red' , 'beta-mannan':'darkgreen' , 'galactan':'yellow' , 
#                                         'chitin':'blue', 'beta-glucan':'blueviolet', 'other':'brown', 
#                                         'starch':'coral', 'cellulose':'darkcyan', 'fructan':'hotpink', 
#                                         'cellooligosaccharide':'indigo', 'alpha-mannan':'lightseagreen', 'rare':'fuchsia' }, 
                   category_orders={"high_level_substr":supervised_data["high_level_substr"].value_counts().keys().tolist()})
fig.update_traces(marker_size=8)
fig.write_html("mongeElkan_3D.html")
fig.show()

In [73]:
supervised_data["high_level_substr"].value_counts()

multiple_substrates           139
mono/di/trisaccharide          78
capsule polysaccharide         60
algal glycans                  40
pectin                         38
xylan                          38
O-antigen                      37
galactan                       36
alpha-glucan                   23
beta-mannan                    19
cellulose                      18
chitin                         15
glycosaminoglycan              14
beta-glucan                    13
N-glycan                       12
exopolysaccharide              10
-                               9
fructan                         8
alpha-mannan                    8
host glycan                     7
xyloglucan                      7
plant polysaccharide            6
glycoprotein                    6
human milk oligosaccharide      5
hemicellulose                   5
O-glycan                        3
Name: high_level_substr, dtype: int64

In [74]:
X_embedded = TSNE(n_components=2, learning_rate='auto').fit_transform(similarities_matrix)

# X_embedded.shape

tsne_data = pd.concat([pd.DataFrame(X_embedded), pd.DataFrame(supervised_data["high_level_substr"])],1)

C:\Users\vedpi\anaconda3\envs\newone\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

C:\Users\vedpi\AppData\Local\Temp/ipykernel_18312/1695015536.py:5: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only



In [75]:
fig = px.scatter(tsne_data, x=0, y=1, color = "high_level_substr", 
                 category_orders={"high_level_substr":supervised_data["high_level_substr"].value_counts().keys().tolist()})
fig.update_traces(marker_size=9)
fig.write_html("mongeElkan_2D.html")
fig.show()